In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from IPython.display import Image
%matplotlib inline
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [3]:
df1 = pd.read_csv('turnstile_180630.txt')
df2 = pd.read_csv('turnstile_180623.txt')


In [4]:
df1['wk'] = 1
df2['wk'] = 0

df =pd.concat([df1,df2],ignore_index=True)
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,wk
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,00:00:00,REGULAR,6667150,2259901,1
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,04:00:00,REGULAR,6667173,2259909,1
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,08:00:00,REGULAR,6667189,2259938,1
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,12:00:00,REGULAR,6667305,2260006,1
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,16:00:00,REGULAR,6667534,2260066,1


In [ ]:
df['DATETIME']=df['DATE'].map(str)+' '+df['TIME']
df['DATETIME']=pd.to_datetime(df['DATETIME'])
df.head()

In [ ]:
# change column name for exits
df.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS',
       'wk', 'DATETIME']


In [ ]:
#Calculate entry diffs
df['ENTRYDIFFS']=(df.sort_values(by=['STATION','UNIT','C/A','SCP','DATETIME']) 
                  .groupby(['STATION','UNIT','C/A','SCP'])['ENTRIES'].diff())

#calculate exit diffs
df['EXITDIFFS']=(df.sort_values(by=['STATION','UNIT','C/A','SCP','DATETIME']) 
                  .groupby(['STATION','UNIT','C/A','SCP'])['EXITS'].diff())

# add weekday labels
df['wkday'] = df['DATETIME'].dt.weekday

# add am pm labels
df['hour'] = df['DATETIME'].dt.hour
df['period'] = ""
df.loc[(df['hour'] >=0) & (df['hour'] <10) , 'period'] = 'Morning'
df.loc[(df['hour'] >=10) & (df['hour'] <19), 'period'] = 'Mid-day'
df.loc[(df['hour'] >=19) & (df['hour'] <=23), 'period'] = 'Evening'
df.head()

In [ ]:
#clear out values don't make sense
df.loc[df['ENTRYDIFFS'] < 0, 'ENTRYDIFFS'] = 0
df.loc[df['ENTRYDIFFS'] > 100000, 'ENTRYDIFFS'] = 0
df.loc[df['EXITDIFFS'] < 0, 'EXITDIFFS'] = 0
df.loc[df['EXITDIFFS'] > 100000, 'EXITDIFFS'] = 0

# calculate total traffic
df['traffic'] = df['ENTRYDIFFS'] + df['EXITDIFFS']

df.head()

In [ ]:
# calculate total traffic by week-weekday for each station

df3=(df.sort_values(by=['STATION','UNIT','C/A','SCP'])
                  .groupby(['STATION','wk','wkday'], as_index=False)['traffic'].sum())

df3.head()

In [ ]:
# calcualte average traffic by weekday-period for each station
df4=(df3.sort_values(by=['STATION'])
                  .groupby(['STATION','wkday'], as_index=False)['traffic'].mean())

df4.head()

In [ ]:
# sort station by traffic in each weekday-period combo

df5=df4.sort_values(by=['wkday','traffic'], ascending=False).groupby(['wkday']).head(10)

# create a new weekday var
df5['wkday1'] = ''
df5.loc[df5['wkday'] ==0 , 'wkday1'] = 'Monday'
df5.loc[df5['wkday'] ==1 , 'wkday1'] = 'Tuesday'
df5.loc[df5['wkday'] ==2 , 'wkday1'] = 'Wednesday'
df5.loc[df5['wkday'] ==3 , 'wkday1'] = 'Thursday'
df5.loc[df5['wkday'] ==4 , 'wkday1'] = 'Friday'
df5.loc[df5['wkday'] ==5 , 'wkday1'] = 'Saturday'
df5.loc[df5['wkday'] ==6 , 'wkday1'] = 'Sunday'



In [ ]:
# make a plot 

plt.figure(figsize = [10,15]) 
plt.subplots_adjust(wspace=1, hspace=1)

plt.subplot(4,2,1) 
df7 = df5.loc[df5['wkday1']=='Monday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Monday')

plt.subplot(4,2,2) 
df7 = df5.loc[df5['wkday1']=='Tuesday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Tuesday')

plt.subplot(4,2,3) 
df7 = df5.loc[df5['wkday1']=='Wednesday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Wednesday')

plt.subplot(4,2,4) 
df7 = df5.loc[df5['wkday1']=='Thursday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Thursday')

plt.subplot(4,2,5) 
df7 = df5.loc[df5['wkday1']=='Friday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Friday')

plt.subplot(4,2,6) 
df7 = df5.loc[df5['wkday1']=='Saturday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Saturday')


plt.subplot(4,2,7) 
df7 = df5.loc[df5['wkday1']=='Sunday'].sort_values(by='traffic', ascending=False).reset_index(drop=True)
plt.barh(df7['STATION'], df7['traffic'])
plt.title('Sunday')




